# From a single split to a decision tree

## Generate some data

In this notebook, we study the internal mechanism used by decision tree to make some classification.

We are going to use a dataset of penguins. Let's first load the dataset and quickly check what type of data do we have to deal with.

In [ ]:
import pandas as pd

dataset = pd.read_csv("datasets/penguins.csv").dropna(subset=["Body Mass (g)"])
dataset.head()

We will frame our supervised learning problem in the following manner: we are interested to predict the penguin species with a single feature, namely the "Body Mass". Then, we will select these data of interest.

In [ ]:
X = dataset["Body Mass (g)"].to_numpy()
y = dataset["Species"].apply(lambda x: x.split()[0]).to_numpy()

To ease some later processing, we are sorting our data.

In [ ]:
import numpy as np

sorted_idx = np.argsort(X)
X = X[sorted_idx]
y = y[sorted_idx]

We can give a look at the class distribution of the dataset.

In [ ]:
import matplotlib.pyplot as plt

for clazz in np.unique(y):
    plt.hist(X[y==clazz], alpha=0.7, label=f'{clazz}', density=True)
plt.legend()
plt.xlabel('Body mass (g)')
plt.ylabel('Class probability');

Since we have only few samples, we can use a `swarmplot` to visualize all samples of the dataset.

In [ ]:
import seaborn as sns

df = pd.DataFrame({'Body mass (g)': X, 'label': y, '': ""})
_ = sns.swarmplot(x='Body mass (g)', y='', hue='label', data=df)

From the graph above, we can observe that predicting the Gentoo penguins vs. the other species will be easy using the body mass. While the Adelie and Chinstrap species are difficult to differentiate. To simplify our illustration, we will therefore only select the Gentoo and Adelie species.

In [ ]:
mask_gentoo = y == "Gentoo"
mask_adelie = y == "Adelie"
X = X[mask_adelie | mask_gentoo]
y = y[mask_adelie | mask_gentoo]

In [ ]:
df = pd.DataFrame({'Body mass (g)': X, 'label': y, '': ""})

In [ ]:
for clazz in np.unique(y):
    plt.hist(X[y==clazz], alpha=0.7, label=f'{clazz}', density=True)
plt.legend()
plt.xlabel('Body mass (g)')
plt.ylabel('Class probability');

## Definition of some concepts: probabilities, entropy and information gain

Decision trees for classification are based on the following concepts: probabilities, entropy, and information gain. We present those concepts and describe where they come into play in the classification procedure.

### Ultimate objective: find the best split

Our goal is to find a threshold which best split the dataset. In other words, we want the boundary such that most samples smaller than this boundary belong to a class and most samples above this boundary belong to the counterpart class. Let's visualise what we mean.

In [ ]:
# create a random state such we all have the same results
rng = np.random.RandomState(42)

In [ ]:
random_idx = rng.choice(np.arange(y.size))

ax = sns.swarmplot(x='Body mass (g)', y='', hue='label', data=df)
ax.set_title(f"random_idx={random_idx}")
_ = ax.plot([X[random_idx], X[random_idx]], [-1, 1], 'r-.')

Use **Ctrl-Enter** to re-execute the previous cell several times in a row.


In the above example, we expect as much as possible blue samples on the left of the decision boundary and as much as possible orange samples on the right of the decision boundary.

Thus, we need to compute some statistics characteristic the quality of this split, sometimes called impurity. Let's group the labels into two groups, all labels corresponding to the samples on the left of the boundary and all labels corresponding to the samples on the right.

<div class="alert alert-success">
    <b>GET SOME INTUITIONS</b>:
     <ul>
      <li>
      Which quantity could you compute to quantify the purity of the new datasets on the left and right side of the split?
      </li>
      <li>
      How could you combine the parent, left, and right purity to have an overall indication of the split quality?
      </li>
    </ul>
</div>

### Probabilities

In [ ]:
random_idx = 128  # restore the first random value to get deterministic results

In [ ]:
ax = sns.swarmplot(x='Body mass (g)', y='', hue='label', data=df)
ax.set_title(f"random_idx={random_idx}")
_ = ax.plot([X[random_idx], X[random_idx]], [-1, 1], 'r-.')

In [ ]:
labels_left = y[:random_idx]
labels_right = y[random_idx:]

Impurity is related to the number of samples of each class on a side of the boundary. We can compute the probability which is the number of samples of a class on the total number of samples.

In [ ]:
from collections import Counter
counter = Counter(labels_left)
counter

In [ ]:
from collections import Counter
def compute_probability(labels):
    counter = Counter(labels)
    n_total_samples = sum(counter.values())
    probabilities = {}
    
    for clazz, count_clazz in counter.items():
        probabilities[clazz] = count_clazz / n_total_samples
    return probabilities

Therefore, for the previous split, we could compute the probabilities for each class on the left and right side of the boundary.

In [ ]:
prob_left = compute_probability(labels_left)
print(f'Probability on the left split {prob_left}')

In [ ]:
prob_right = compute_probability(labels_right)
print(f'Probability on the right split {prob_right}')

Those results make sense. On the left side of the boundary, we mainly have Adelie penguin and very few Gentoo. On the right side of the partition, the separation is less ovious and the probability are not close to 0 and 1.

Therefore, we can have the intuition that we need a split which will maximize the probability of a class on each side of the partition.

### Entropy

In the previous section, we computed probabilities. It gives several statistics depending on the class, and it will be handy to get a single statistic which characterises the impurity of a side of the boundary: we will use the entropy definition.

Entropy is defined: $H(X) = - \sum_{k=1}^{K} p(X_k) \log_{2} p(X_k)$, where $K$ is the number of classes. In practise the entropy function looks like:

![title](https://upload.wikimedia.org/wikipedia/commons/2/22/Binary_entropy_plot.svg)

Considering some labels, the entropy will be maximum if there are as many samples from one class than another. It will be minimum when there are only samples from a single class.

In [ ]:
def entropy(labels):
    probabilities = compute_probability(labels)

    H_X = 0
    for clazz in probabilities:
        H_X -= probabilities[clazz] * np.log2(probabilities[clazz])

    return H_X

We can compute the entropy for each side of the split.

In [ ]:
entropy_left = entropy(labels_left)
print(f'Entropy on the left split {entropy_left}')

In [ ]:
entropy_right = entropy(labels_right)
print(f'Entropy on the right split {entropy_right}')

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>
      What would be the entropy of the full dataset given the fact that the two classes are balanced?
      </li>
      <li>
      Check the correctness of your answer using the <tt>entropy</tt> function.
      </li>
    </ul>
</div>

We see that the entropy on the right side is small. We only had orange samples on that side. The entropy of the left side is closer to one because we have almost half-half orange and blue samples.

### Information gain

For the current split, we currently have two statistics, an entropy for each side of the split. It will be handy to get a single statistic to summarise the quality of the split. It is called the information gain.

Indeed, the information gain corresponds to the entropy computed before splitting the data to which we subtract the entropy from the right and left side of the split (with some additional normalisation factor).

In [ ]:
def information_gain(labels_parent, labels_left, labels_right):
    entropy_parent = entropy(labels_parent)
    entropy_left = entropy(labels_left)
    entropy_right = entropy(labels_right)

    n_samples_parent = labels_parent.size
    n_samples_left = labels_left.size
    n_samples_right = labels_right.size

    normalized_entropy_left = ((n_samples_left / n_samples_parent) * 
                               entropy_left)
    normalized_entropy_right = ((n_samples_right / n_samples_parent) *
                                entropy_right)

    return (entropy_parent -
            normalized_entropy_left - normalized_entropy_right)

In [ ]:
info_gain_split = information_gain(y, labels_left, labels_right)

In [ ]:
print(f'Information for split at index {random_idx}:'
      f' {info_gain_split:.3f}')

## Meaning of information gain

This metric is called information gain because it relates to the gain in "purity" obtained by partitioning the data. We can try several random partitions and see that a higher is the gain corresponds to a better split to partition the two classes.

In [ ]:
random_idx = rng.choice(np.arange(y.size))
labels_left = y[:random_idx]
labels_right = y[random_idx:]

ax = sns.swarmplot(x='Body mass (g)', y='', hue='label', data=df)
_ = ax.plot([X[random_idx], X[random_idx]], [-1, 1], '-.')

info_gain_split = information_gain(y, labels_left, labels_right)
print(f'Information for split at index {random_idx}:'
      f' {info_gain_split:.3f}')

Press **Ctrl-Enter** to execute the previous cell several times in a row to see how the information gain evolves.

<div class="alert alert-success">
    <b>EXERCISE: Brute force search for the maximum information gain</b>:
    <br>
    Instead of trying to random split, we can use a greedy strategy by iteratively trying all possible value and use the one maximum the information gain.
     <ul>
      <li>
      Iterate over all possible split indices in the dataset and store the information gain (note, the values in arrays X and y are already pre-sorted by increasing feature value in X);
      </li>
      <li>
      Find the index corresponding to the maximum information gain;
      </li>
      <li>
      Plot the all information gain values for all possible threshold;
      </li>
      <li>
      Plot a marker for the threshold maximizing the information gain;
      </li>
      <li>
      Plot the original swarm plot and superpose the line corresponding to the best threshold found.
      </li>
    </ul>
</div>

<div class="alert alert-success">
    <b>EXERCISE: DecisionTreeClassifier as a decision stump</b>:
    <ul>
      <li>
          Use a <tt>sklearn.tree.DecisionTreeClassifier</tt> to classifiy the data X and y. Train the decision tree. Ensure to use the good criterion and to limit the depth of the tree to get a similar process than what we did before.
      </li>
    </ul>
</div>

Read the online documentation for [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) to learn more about its constructor parameters.

In [ ]:
X.shape

In [ ]:
X_reshaped = X[:, np.newaxis]

In [ ]:
X_reshaped.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# TODO
# tree = DecisionTreeClassifier()

Once this classifier is trained, we can plot the resulting decision and compare it with the previous procedure.

In [ ]:
from sklearn.tree import plot_tree

_ = plot_tree(tree)